In [1]:
#import sys
#!{sys.executable} -m pip install tensorflow

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import image

import examplesetup
import lib.deconvolution as deconvolution
import lib.utils as utils
import lib.xtrace as xtrace
import lib.datagen as datagen

from numpy.random import default_rng
rng = default_rng()

import tensorflow as tf
from tensorflow import keras
import numpy as np


img_shape = (100, 100)

class SyntheticDepthBlur(keras.utils.Sequence):

    #could include options to configure config params (or there ranges)
    def __init__(self, batch_size, batches, img_shape):
        self.batch_size = batch_size
        self.img_shape = img_shape
        self.batches = batches

    def __len__(self):
        return self.batches

    def __getitem__(self, idx):
        
        data = np.array([
            datagen.get_synthetic_data_pair()
            for i in range(self.batch_size)
        ])
        distorted_imgs = data[:,0, :, :, np.newaxis]
        images = data[:,1, :, :, np.newaxis]
        return (distorted_imgs, images)

#test = SyntheticDepthBlur(50, 5, (100, 100))
#v = test[0]
#plt.imshow(v[1][0,:,:,0])
#plt.show()

bshuf filter already loaded, skip it.


In [3]:
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, Conv2DTranspose, MaxPooling2D, ZeroPadding2D
from keras import backend as keras_backend

class modelsClass(object):

    def __init__(self, img_rows = 272, img_cols = 480):

        self.img_rows = img_rows
        self.img_cols = img_cols

    def addPadding(self, layer, level): #height, width, level):
    
        w1, h1 = self.img_cols, self.img_rows
        w2, h2 = int(w1/2), int(h1/2)
        w3, h3 = int(w2/2), int(h2/2)
        w4, h4 = int(w3/2), int(h3/2)
        h = [h1,h2,h3,h4]
        w = [w1,w2,w3,w4]
        
        # Target width and height
        tw = w[level-1]
        th = h[level-1]
        
        # Source width and height
        lsize = keras_backend.int_shape(layer)
        sh = lsize[1]
        sw = lsize[2]

        pw = (0, tw - sw)
        ph = (0, th - sh)

        layer = ZeroPadding2D(padding=(ph,pw),data_format="channels_last")(layer)
    
        return layer
        
    def getDeepGyro(self):

        input_blurred = Input((self.img_rows, self.img_cols,1))
        input_blurx = Input((self.img_rows, self.img_cols,1))
        input_blury = Input((self.img_rows, self.img_cols,1))
        
        inputs = concatenate([input_blurred,input_blurx,input_blury])
        
        conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
        conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
        conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

        conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
        conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

        conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
        conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

        conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)
        conv5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(conv5)

        up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
        up6 = self.addPadding(up6,level=4)
        up6 = concatenate([up6,conv4], axis=3)
        conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(up6)
        conv6 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)

        up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
        up7 = self.addPadding(up7,level=3)
        up7 = concatenate([up7,conv3], axis=3)
        conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(up7)
        conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

        up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
        up8 = self.addPadding(up8,level=2)
        up8 = concatenate([up8,conv2], axis=3)
        conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(up8)
        conv8 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)

        up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
        up9 = self.addPadding(up9,level=1)
        up9 = concatenate([up9,conv1], axis=3)
        conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(up9)
        conv9 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)

        conv10 = Conv2D(3, (1, 1), activation='linear')(conv9)
        
        model = Model(inputs=[input_blurred,input_blurx,input_blury], outputs=conv10)
           
        #adam = optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        #model.compile(optimizer = adam, loss = 'mean_squared_error')

        return model
        
    def getDeepBlind(self):

        input_blurred = Input((self.img_rows, self.img_cols,1))
        
        m = 1
        conv1 = Conv2D(64*m, (3, 3), activation='relu', padding='same')(input_blurred)
        conv1 = Conv2D(64*m, (3, 3), activation='relu', padding='same')(conv1)
        pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

        conv2 = Conv2D(128*m, (3, 3), activation='relu', padding='same')(pool1)
        conv2 = Conv2D(128*m, (3, 3), activation='relu', padding='same')(conv2)
        pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

        conv3 = Conv2D(256*m, (3, 3), activation='relu', padding='same')(pool2)
        conv3 = Conv2D(256*m, (3, 3), activation='relu', padding='same')(conv3)
        pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

        conv4 = Conv2D(512*m, (3, 3), activation='relu', padding='same')(pool3)
        conv4 = Conv2D(512*m, (3, 3), activation='relu', padding='same')(conv4)
        pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

        conv5 = Conv2D(1024*m, (3, 3), activation='relu', padding='same')(pool4)
        conv5 = Conv2D(1024*m, (3, 3), activation='relu', padding='same')(conv5)

        up6 = Conv2DTranspose(512*m, (2, 2), strides=(2, 2), padding='same')(conv5)
        up6 = self.addPadding(up6,level=4)
        up6 = concatenate([up6,conv4], axis=3)
        conv6 = Conv2D(512*m, (3, 3), activation='relu', padding='same')(up6)
        conv6 = Conv2D(512*m, (3, 3), activation='relu', padding='same')(conv6)

        up7 = Conv2DTranspose(256*m, (2, 2), strides=(2, 2), padding='same')(conv6)
        up7 = self.addPadding(up7,level=3)
        up7 = concatenate([up7,conv3], axis=3)
        conv7 = Conv2D(256*m, (3, 3), activation='relu', padding='same')(up7)
        conv7 = Conv2D(256*m, (3, 3), activation='relu', padding='same')(conv7)

        up8 = Conv2DTranspose(128*m, (2, 2), strides=(2, 2), padding='same')(conv7)
        up8 = self.addPadding(up8,level=2)
        up8 = concatenate([up8,conv2], axis=3)
        conv8 = Conv2D(128*m, (3, 3), activation='relu', padding='same')(up8)
        conv8 = Conv2D(128*m, (3, 3), activation='relu', padding='same')(conv8)

        up9 = Conv2DTranspose(64*m, (2, 2), strides=(2, 2), padding='same')(conv8)
        up9 = self.addPadding(up9,level=1)
        up9 = concatenate([up9,conv1], axis=3)
        conv9 = Conv2D(64*m, (3, 3), activation='relu', padding='same')(up9)
        conv9 = Conv2D(64*m, (3, 3), activation='relu', padding='same')(conv9)

        conv10 = Conv2D(1, (1, 1), activation='linear')(conv9)
        
        model = Model(inputs=input_blurred, outputs=conv10)
        
        #adam = optimizers.Adam(lr=0.0000125, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        #model.compile(optimizer = adam, loss = 'mean_squared_error')

        return model

In [ ]:
# Free up RAM in case the model definition cells were run multiple times
keras.backend.clear_session()

batch_size = 32
batches = 100

# Build model
models = modelsClass(*img_shape)
model = models.getDeepBlind()
#model.summary()
train_gen = SyntheticDepthBlur(batch_size, batches, img_shape)
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss=tf.keras.losses.MeanAbsolutePercentageError())
callbacks = [
    keras.callbacks.ModelCheckpoint("trained_networks/deconvolve.h5", monitor="loss", save_best_only=True)
]

epochs = 500
model.fit(train_gen, epochs=epochs, callbacks=callbacks)


Epoch 1/500


2022-04-08 15:24:12.677446: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_2 in the registry.
Traceback (most recent call last):

  File "/opt/conda/envs/hdf5-kernel-environment/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 259, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_2 in the registry.


2022-04-08 15:24:12.677507: W tensorflow/core/kernels/data/generator_dataset_op.cc:107] Error occurred when finalizing GeneratorDataset iterator: INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_2 in the registry.
Traceback (most recent call last):

  File "/opt/conda/envs/hdf5-kernel-environment/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 259, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback 

100/100 [==============================] - 35s 325ms/step - loss: 4250314.0000
Epoch 2/500
100/100 [==============================] - 32s 323ms/step - loss: 1220741.3750
Epoch 3/500
100/100 [==============================] - 32s 324ms/step - loss: 569960.4375
Epoch 4/500
100/100 [==============================] - 32s 323ms/step - loss: 475592.1875
Epoch 5/500
100/100 [==============================] - 31s 312ms/step - loss: 25247692.0000
Epoch 6/500
100/100 [==============================] - 31s 313ms/step - loss: 555317.3125
Epoch 7/500
100/100 [==============================] - 32s 322ms/step - loss: 455616.1562
Epoch 8/500
100/100 [==============================] - 32s 325ms/step - loss: 445349.7500
Epoch 9/500
100/100 [==============================] - 31s 313ms/step - loss: 3415954.0000
Epoch 10/500
100/100 [==============================] - 31s 312ms/step - loss: 445936.4062
Epoch 11/500
100/100 [==============================] - 31s 313ms/step - loss: 4237675.5000
Epoch 12/500
1

In [ ]:

#Test combinations:
# With/without image noise in training
# With/without changing pixel hit position
# With/without chaning config parameters
# deepBlind/DeepGyro with extra data
#

%matplotlib widget

model = n
img = image.imread("example_image.png").sum(axis=2)
distorted_img, img = datagen.get_synthetic_data_pair(img_shape, G=G_glob)
recovered_img = model(distorted_img[np.newaxis,:,:], training=False).numpy()[0,:,:,0]
fig, axs = plt.subplots(1, 3, figsize=(15, 10), sharex=True, sharey=True)
perc = np.percentile(distorted_img, 99)
axs[0].imshow(img, vmax=perc, vmin=0)
axs[1].imshow(distorted_img,  vmax=perc, vmin=0)
axs[2].imshow(recovered_img,  vmax=perc, vmin=0)

In [ ]:
model.summary()